In [6]:
import pandas as pd
import statistics
from tqdm import tqdm
import ast

In [ ]:
all_farmers = pd.read_csv("../../kornmo-data-files/raw-data/farm-information/farmers-with-coordinates-and-soil_quality.csv", usecols=['orgnr', 'longitude', 'latitude', 'elevation'])
print(all_farmers.columns)

soil_qualities = pd.read_csv("../../kornmo-data-files/raw-data/soil-data/jordsmonn.csv", usecols=['id', 'JORDKVALIT', 'JORDKVALITET'])
print(soil_qualities.columns)

farmers_fields = pd.read_csv("../../kornmo-data-files/raw-data/farm-information/orgnrs_per_field.csv", usecols=['field_id', 'municipal_nr', 'orgnrs'])
print(farmers_fields.columns)
print(farmers_fields.head())

In [ ]:
farmers_missing_fields = []
farmers_soil_quality = {}


p_bar = tqdm(total=len(farmers_fields), iterable=farmers_fields.iterrows())
for field_index, field in p_bar:

    soil_quality = soil_qualities.loc[field['field_id'], 'JORDKVALITET']
    all_orgs = ast.literal_eval(field['orgnrs'])

    for farmer in all_orgs:
        farmer = int(farmer)
        if farmer not in farmers_soil_quality:
            farmers_soil_quality[farmer] = []
        farmers_soil_quality[int(farmer)].append(int(soil_quality))

p_bar = tqdm(total=len(all_farmers), iterable=all_farmers.iterrows())
for farm_index, farmer in p_bar:
    orgnr = farmer['orgnr']
    if orgnr in farmers_soil_quality:
        qualities = farmers_soil_quality[orgnr]

        all_farmers.loc[farm_index, 'mean_soil_quality'] = statistics.mean(qualities)
        all_farmers.loc[farm_index, 'soil_quality_1'] = int(qualities.count(1))
        all_farmers.loc[farm_index, 'soil_quality_2'] = int(qualities.count(2))
        all_farmers.loc[farm_index, 'soil_quality_3'] = int(qualities.count(3))

    else:
        farmers_missing_fields.append(orgnr)

print(f"There are {len(farmers_missing_fields)} farmers without a field in orgnrs_per_field.csv")

if all_farmers.isna().sum().sum() != 0:
    print(f"There are {all_farmers.isna().sum().sum()} NaN values in the dataset")
    print(all_farmers.isna().sum())

all_farmers.to_csv("../../kornmo-data-files/raw-data/farm-information/farmers-with-coordinates-and-soil_quality.csv")

